In [15]:
from ipywidgets import interact, interactive, fixed, interact_manual, Layout
import ipywidgets as widgets
import numpy as np
import pandas as pd
import geopandas as gpd
import folium
from folium import plugins
import ipywidgets
import geocoder
import geopy
import plotly.express as px
import voila
import openpyxl

In [16]:
df = pd.read_excel('tmp.xlsx')

In [17]:
df.head(10)

,КД,ФИО,СрокПЗ,КП,Сотрудник,ЗаКем,адрес,latitude,longitude
0,1КД,Пташинский,1,10,Воробьев,Восток,"628606, Ханты-Мансийский Автономный округ - Юг...",60.937704,76.565765
1,2КД,Хайлова,10,4,Щербаков,Запад,"620920, Свердловская обл, г Екатеринбург, посе...",56.868716,60.291325
2,3КД,Щанкин,20,5,Воробьев,Восток,"403992, Волгоградская обл, Новоаннинский р-н, ...",50.423203,43.125276
3,4КД,Сильнягин,30,20,Щербаков,Запад,"346621, Ростовская обл, Багаевский р-н, хутор ...",47.308725,40.501115
4,5КД,Рахматулина,40,40,Воробьев,Восток,"357736, Ставропольский край, г Кисловодск, ул ...",43.922831,42.708087
5,6КД,Сабадаш,50,80,Щербаков,Запад,"614520, Пермский край, Пермский р-н, село Култ...",57.897729,55.941044
6,7КД,Алексеев,60,100,Кондрашов,ИП,"162604, Вологодская обл, г Череповец, ул Парти...",59.149419,37.941283
7,8КД,Каримова,65,200,Кондрашов,ИП,"663508, Красноярский край, Манский р-н, село Н...",55.785271,93.708201
8,9КД,Белянская,70,400,Кондрашов,ИП,"628449, Ханты-Мансийский Автономный округ - Юг...",61.615845,72.160523
9,10КД,Бутов,100,300,Кондрашов,ИП,"347323, Ростовская обл, Цимлянский р-н, г Цимл...",47.651767,42.083286


In [18]:
originality = widgets.SelectMultiple(
    options = list(df['ЗаКем'].unique()),
    value = list(df['ЗаКем'].unique()),
    description='ЗаКем',
    disabled=False,
    rows=len(pd.unique(df['ЗаКем']))
)


In [19]:
worker = widgets.SelectMultiple(
    options = list(df['Сотрудник'].unique()),
    value = list(df['Сотрудник'].unique()),
    description='Сотрудник',
    disabled=False,
    rows=len(pd.unique(df['Сотрудник']))
)

In [20]:
op = widgets.SelectionRangeSlider(
    options=df['СрокПЗ'],
    description='Срок просроченной задолженности, дни',
    disabled=False,
    style={'background-color': '#000'}
)

In [21]:
oa = widgets.SelectionRangeSlider(
    options=df['КП'],
    description='Размер кредитного портфеля, рубли',
    disabled=False,
    style={'background-color': '#000'}
)

In [22]:
def generate_color(kp):
    if kp <= 10:
        c_outline, c_fill = '#FFC0CB', '#FFC0CB'
        m_opacity, f_opacity = 1, 1
    else:
        c_outline, c_fill = '#c0392b', '#e74c3c'
        m_opacity, f_opacity = 1, 1
    return c_outline, c_fill, m_opacity, f_opacity

In [23]:
def generate_popup(КД, ФИО):
    return f'''<strong>Номер КД:</strong> {КД}<br><strong>ФИО:</strong> {ФИО}'''

In [24]:
def change_parameters(originality, worker, op, oa):

    low_op = op[0]
    high_op = op[1]
    
    low_oa = oa[0]
    high_oa = oa[1]
    
    df_update = df[df['ЗаКем'].isin(originality)]
    df_update1 = df_update[df_update['Сотрудник'].isin(worker)]
    df_update2 = df_update1[df_update1['СрокПЗ'].between(low_op,high_op)]
    df_update3 = df_update2[df_update2['КП'].between(low_oa,high_oa)]
    
    m_tmp = folium.Map(location=[63.391522, 96.328125], zoom_start=3,  control_scale = True)
  
    for _, row in df_update3.iterrows():
        c_outline, c_fill, m_opacity, f_opacity = generate_color(row['КП'])
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            popup=generate_popup(row['КД'], row['ФИО']),
            color=c_outline,
            fill=True,
            fillColor=c_fill,
            opacity=m_opacity,
            fillOpacity=f_opacity,
            radius=0.1
        ).add_to(m_tmp)
    display(m_tmp)

In [25]:
out = widgets.interactive_output(
        change_parameters, 
          {'originality': originality,
           'worker': worker, 
           'op': op, 
           'oa': oa
          }
      )
ui = widgets.HBox(
        [originality, 
         worker,
          op, 
          oa
        ], 
        layout=Layout(display='flex', flex_flow='row wrap', justify_content='space-between')
    )

In [26]:
display(ui, out)

Output(outputs=({'output_type': 'error', 'ename': 'IndexError', 'evalue': 'tuple index out of range', 'traceba…